In [10]:
from datetime import datetime, timedelta
import time
import pandas as pd
import pandas_market_calendars as mcal
import ta
from alpaca_trade_api.rest import REST, TimeFrame
import os
from sklearn.preprocessing import MinMaxScaler, maxabs_scale


In [11]:
FMP_API_KEY = "45d2c43abf8e1e4ba47884a88f378027"
POLYGON_KEY = "AVs1KTDp0aDoioS_EQ2KN63JiIyTaKLf"
FINNHUB_KEY = "ceivrvqad3if39n22g3gceivrvqad3if39n22g40"
TWELVEDATA_KEY = "deb706049cdd47058e1210eacc3aefa5"
ALPHA_VANTAGO_KEY = "PX771WNGSYWHEKLW"
NASDAQ_DATA_LINK_KEY = 'ouH2XbhJ1k9zz-6KtKPz'
ALPACA_API_KEY='PKNHYYQ7OUDKJK3NP6EX'
ALPACA_S_KEY='zo3qv6aUJJI14aSY2AgUOfuDufcapDx6yFBBixGD'
ALPACA_BASE_URL = 'https://data.alpaca.markets/v2'

In [12]:
DATASET_DIR = r'D:\repo\stonks\data\dataset'
api = REST(ALPACA_API_KEY, ALPACA_S_KEY, ALPACA_BASE_URL, api_version='v2')


def raw_csv_path(sym):
    # return fr'D:\repo\stocks\stonkAI\data\dataset\{sym}.csv'
    return os.path.join(DATASET_DIR, f'{sym}.csv')


def data_csv_path(sym):
    return os.path.join(DATASET_DIR, f'{sym}_data.csv')


def file_path(sym, type, file_type='csv', dir=DATASET_DIR):
    return os.path.join(dir, f'{sym}_{type}.{file_type}')


def add_unix(df):
    df['timestamp'] = df.index
    df['unixTime'] = [time.mktime(ts.timetuple()) for ts in df['timestamp']]
    return df


def get_last_available_day(timestamp_str: str):
    '''fetches the date of the last available trading day'''
    timestamp = datetime.strptime(timestamp_str, '%Y-%m-%d')
    day_before = timestamp - timedelta(days=1)
    timestamp_str = day_before.strftime('%Y-%m-%d')
    return timestamp_str


def create_output(data, num_days=5):
    # Create output columns for high and low range of 15 days
    data_points_per_day = 447
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=num_days *
                                                        data_points_per_day)
    data[f'highest_in_{num_days}_days'] = data['high'].rolling(
        window=indexer, min_periods=1).max()
    data[f'lowest_in_{num_days}_days'] = data['low'].rolling(
        window=indexer, min_periods=1).min()
    return data


def fetch_new_data(sym, start_date, end_date=None):
    '''
    Grabs last recorded date from CSV and fetches new data from that date to today
    Saves all datasets and latest scaled entry with corresponding timestamp
    '''
    tf = TimeFrame.Minute
    adj = 'raw'
    mode = 'w'
    save = True
    header = True
    old_data = pd.DataFrame()
    try:
        old_data = pd.read_csv(raw_csv_path(sym))
        # last_recorded_timestamp = old_data.iloc[-1]['timestamp'][:10]
        start_date = old_data.iloc[-1]['timestamp'][:10]
        start_date = get_last_available_day(start_date)
        print(f'recorded day before start date: {start_date}')

    except Exception as e:
        print(f'ERROR: {e}')
        print('no old data found')
    # get new data here
    # print('fetching new data')
    data = api.get_bars(sym,
                        tf,
                        start=start_date,
                        end=end_date,
                        adjustment=adj).df
    data = add_unix(data)
    print(data.shape)
    print(data.head(1))
    print(data.tail(1))
    if data.empty:
        return
    print('cleaning raw data')
    all_data = pd.concat([data,
                          old_data]).drop_duplicates().reset_index(drop=True)
    all_data.sort_values(by=['unixTime'], inplace=True)
    all_data = all_data[all_data.columns.drop(
        list(all_data.filter(regex='Unnamed')))]
    # TODO validate data sparsity

    print('creating additional data')
    # create outputs
    all_data_with_outputs = create_output(all_data)

    # grab technical indicators
    all_data_with_ta_and_outputs = ta.add_all_ta_features(
        all_data_with_outputs,
        open="open",
        high="high",
        low="low",
        close="close",
        volume="volume")

    # all_data_with_ta_and_outputs
    # remove first 50 rows to remove null values from technical indicators
    all_data_with_ta_and_outputs = all_data_with_ta_and_outputs.iloc[
        50:].reset_index(drop=True)
    last_time_stamp = all_data_with_ta_and_outputs.iloc[-1]['timestamp']
    last_unix = all_data_with_ta_and_outputs.iloc[-1]['unixTime']
    # clean up data
    print('cleaning up data')
    # normalize outputs by dividing by close
    all_data_with_ta_and_outputs[
        'highest_in_5_days_percent'] = all_data_with_ta_and_outputs[
            'highest_in_5_days'] / all_data_with_ta_and_outputs['close']
    all_data_with_ta_and_outputs[
        'lowest_in_5_days_percent'] = all_data_with_ta_and_outputs[
            'lowest_in_5_days'] / all_data_with_ta_and_outputs['close']
    # remove columns timestamp, unixTime, and the index
    outputs = all_data_with_ta_and_outputs[[
        'close', 'highest_in_5_days', 'highest_in_5_days_percent',
        'lowest_in_5_days', 'lowest_in_5_days_percent'
    ]]
    inputs = all_data_with_ta_and_outputs.drop(columns=[
        'timestamp', 'unixTime', 'highest_in_5_days', 'lowest_in_5_days',
        'highest_in_5_days_percent', 'lowest_in_5_days_percent'
    ])
    inputs.dropna(inplace=True, axis=1)
    print('inputs.shape', inputs.shape)
    print(inputs.columns)
    scaled_inputs = maxabs_scale.scale(inputs)
    # save data to csv
    if save:
        print('saving to csv')
        all_data.to_csv(raw_csv_path(sym), mode=mode, header=header)
        all_data_with_ta_and_outputs.to_csv(data_csv_path(sym),
                                            mode=mode,
                                            header=header)
        inputs.to_csv(file_path(sym, 'inputs_raw'), mode=mode, header=header)
        scaled_inputs.to_csv(file_path(sym, 'inputs_scaled'),
                             mode=mode,
                             header=header)
        outputs.to_csv(file_path(sym, 'outputs_raw'), mode=mode, header=header)
        # savve last entry of scaled_inputs to csv
        last_scaled_input = scaled_inputs.iloc[-1:]
        last_scaled_input.to_csv(file_path(sym, 'last_input_entry', 'csv'),
                                 mode=mode,
                                 header=header)
        with open(file_path(sym, 'last_unix', 'txt'), 'w') as f:
            f.write(str(last_unix))
    print('done')
    print(last_time_stamp)
    return last_time_stamp, last_scaled_input


sym = 'AAPL'
start_date = '2000-01-01'
end_date = '2023-01-26'
fetch_new_data(sym, start_date=start_date, end_date=end_date)

recorded start date: 2023-01-24
2023-01-23
recorded day before start date: 2023-01-23
fetching new data
(3250, 9)
                            open   high     low   close  volume  trade_count  \
timestamp                                                                      
2023-01-23 09:00:00+00:00  138.5  138.5  137.75  137.75    3970          148   

                                 vwap                 timestamp      unixTime  
timestamp                                                                      
2023-01-23 09:00:00+00:00  137.942632 2023-01-23 09:00:00+00:00  1.674493e+09  
                             open    high     low   close  volume  \
timestamp                                                           
2023-01-27 00:59:00+00:00  143.67  143.69  143.66  143.69    7243   

                           trade_count        vwap                 timestamp  \
timestamp                                                                      
2023-01-27 00:59:00+00:00           4

c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
c:\Users\Michael\AppData\Local\Programs\Python\Python39\lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


cleaning up data
saving to csv
done
2023-01-27 00:59:00+00:00
